## Lendo e sepando colunas

In [12]:
import pandas as pd

df = pd.read_csv('risco_cretido.csv')
previsores = df.iloc[:, 0:4].values
classe = df.iloc[:, 4].values

## Transformação de variáveis categóricas

In [13]:
from sklearn.preprocessing import LabelEncoder
labelenconder = LabelEncoder()

previsores[:,0] = labelenconder.fit_transform(previsores[:,0])
previsores[:,1] = labelenconder.fit_transform(previsores[:,1])
previsores[:,2] = labelenconder.fit_transform(previsores[:,2])
previsores[:,3] = labelenconder.fit_transform(previsores[:,3])

## Naive Bayes -> Tabela de Probabilidade

In [3]:
from sklearn.naive_bayes import GaussianNB
classificador = GaussianNB()
classificador.fit(previsores, classe)
resultado = classificador.predict([[0, 0, 1, 2], [3, 0, 0, 0]])
print(resultado)

['baixo' 'moderado']


In [4]:
print(classificador.classes_)
print(classificador.class_count_)
print(classificador.class_prior_)

['alto' 'baixo' 'moderado']
[6. 5. 3.]
[0.42857143 0.35714286 0.21428571]


## Árvore de Decisão

In [5]:
from sklearn.tree import DecisionTreeClassifier, export
classificador = DecisionTreeClassifier(criterion='entropy')
classificador.fit(previsores, classe)

print(classificador.feature_importances_)

export.export_graphviz(classificador,
                       out_file = 'arvore.dot',
                       feature_names = ['Historia', 'Divida', 'Garantias', 'Renda'],
                       class_names = ['Alto', 'Moderado', 'Baixo'],
                       filled = True)

[0.41607015 0.10294068 0.03885431 0.44213486]


/home/lindino/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.tree.export module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


## Regras -> CN2

In [6]:
import Orange

base = Orange.data.Table('risco_cretido.csv')
print(base.domain)

cn2_learner = Orange.classification.rules.CN2Learner()
classificador = cn2_learner(base)

for regras in classificador.rule_list:
    print(regras, end="\n")

resultado = classificador([['boa', 'alta', 'nenhuma', 'acima_35'], ['ruim', 'alta', 'adequada', '0_15']])

print('\nResultados: ', end=' ')
for i in resultado:
    print(base.domain.class_var.values[i], end=' ')

[historia, divida, garantias, renda | risco]
IF renda==0_15 THEN risco=alto 
IF historia==boa AND divida!=alta THEN risco=baixo 
IF historia==boa AND garantias!=nenhuma THEN risco=baixo 
IF historia==boa AND renda!=15_35 THEN risco=baixo 
IF historia==boa THEN risco=moderado 
IF divida==alta THEN risco=alto 
IF historia!=desconhecida THEN risco=moderado 
IF garantias==adequada THEN risco=baixo 
IF renda==15_35 THEN risco=moderado 
IF historia==desconhecida THEN risco=alto 
IF TRUE THEN risco=alto 

Resultados:  baixo alto 

## Regressão Logística

In [14]:
from sklearn.linear_model import LogisticRegression
classificador = LogisticRegression()
classificador.fit(previsores, classe)

print(classificador.intercept_)
print(classificador.coef_)

[ 0.39746278 -0.51968744  0.12222466]
[[ 0.4996621  -0.38257429  0.46670496 -0.85049945]
 [-0.51993576  0.14838452 -0.25100572  1.03221549]
 [ 0.02027366  0.23418977 -0.21569924 -0.18171604]]


In [17]:
resultado = classificador.predict([[0, 0, 1, 2], [3, 0, 0, 0]])
resultado2 = classificador.predict_proba([[0, 0, 1, 2], [3, 0, 0, 0]])
print('Resultado: {}\n Probabilidade: {}'.format(resultado, resultado2))

Resultado: ['baixo' 'alto']
 Probabilidade: [[0.09189631 0.77373529 0.13436841]
 [0.83401941 0.01564753 0.15033306]]
